In [137]:
# import classes from folder app/classes.py
from app.libs.classes import Shift, Nurse
import json


In [138]:
with open('instances/n_nurses_9_m_12_y_2022/shifts_12-2022_1.json', 'r') as f:
    shifts = json.load(f)

shifts = {shift['shift_id']: Shift(**shift) for shift in shifts}

with open('instances/n_nurses_9_m_12_y_2022/n_nurses_9_12-2022_1.json', 'r') as f:
    nurses = json.load(f)

nurses = {nurse['nurse_id']: Nurse(**nurse) for nurse in nurses}
nurses
# Additional parameters 

{0: Nurse(nurse_id=0, nurse_name='Nurse 0', shift_preference='afternoon', accumulated_hours=8, morning_availability_labor_day=0, morning_availability_weekend=0, afternoon_availability_labor_day=0, afternoon_availability_weekend=0, dates_off=[31, 11, 24, 25, 18, 10, 17], vacations=[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]),
 1: Nurse(nurse_id=1, nurse_name='Nurse 1', shift_preference='morning', accumulated_hours=-9, morning_availability_labor_day=0, morning_availability_weekend=0, afternoon_availability_labor_day=1, afternoon_availability_weekend=1, dates_off=[11, 17], vacations=[]),
 2: Nurse(nurse_id=2, nurse_name='Nurse 2', shift_preference='morning', accumulated_hours=-29, morning_availability_labor_day=0, morning_availability_weekend=1, afternoon_availability_labor_day=0, afternoon_availability_weekend=0, dates_off=[31], vacations=[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]),
 3: Nurse(nurse_id=3, nurse_name='Nurse 3', shift_preference='afternoon', accumulated_

In [139]:
DM = 8
HMM = 216
# Modelo

In [140]:
from datetime import datetime, timedelta
from math import floor
from typing import Dict

from app.libs.classes import Shift, Nurse
from app.libs.constants import HMM, DM
import pulp as plp
from itertools import product

In [141]:
import pulp as plp

model = plp.LpProblem("Nurse Scheduling", plp.LpMinimize)
I = set(nurses.keys())
J = set(shifts.keys())
K = set([datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day for j in J])
week=set([datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').isocalendar()[1] for j in J])

valid_keys = [(i, j) for (i,j) in product(I, J) if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day not in nurses[i].vacations]


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [142]:
X = plp.LpVariable.dicts("X", valid_keys, cat=plp.LpBinary)
W = plp.LpVariable.dicts("W", [(i, k) for (i,k) in product(I, K)], cat=plp.LpBinary)
Zmorning=plp.LpVariable.dicts("Zmorning", [(i, w) for (i,w) in product(I, week)], cat=plp.LpBinary)
Zafternoon=plp.LpVariable.dicts("Zafternoon", [(i, w) for (i,w) in product(I, week)], cat=plp.LpBinary)

V = plp.LpVariable.dicts("V", I, cat=plp.LpContinuous,lowBound=0)  # overtime
Y = plp.LpVariable.dicts("Y", I, cat=plp.LpContinuous,lowBound=0) # TOTAL NUMBER OF SHIFTS ASSIGNED TO NURSE i
MDH = plp.LpVariable("MDH", cat=plp.LpContinuous,lowBound=0) # MAXIMUM DIFFERENCE BETWEEN MAXIMUM SHIFTS AND REAL NUMBER OF SHIFTS ASSIGNED
DOff = plp.LpVariable.dicts("DOff", I, cat=plp.LpContinuous,lowBound=0) 


# Objective function

In [143]:
expr_PDL = plp.lpSum([(X[i, j] for (i,j) in valid_keys if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day in nurses[i].dates_off)])
# accounts for the number of weekends a person is assigned to a shif not in his/her preference
expr_PWE = plp.lpSum([(X[i, j]) for (i,j) in valid_keys if
                        datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').weekday() in [5, 6] and
                        (nurses[i].shift_preference != shifts[j].shift_type)])

Obj_2 = MDH + plp.lpSum([V[i] for i in I]) * (1 / DM)

In [144]:
# accounts the number of shifts assigned to a nurse
for nurse in I:
    model+=Y[nurse] == plp.lpSum([X[nurse, shift] for shift in J if (nurse, shift) in valid_keys]), f"assigned_shifts_{nurse}"

In [145]:
 # accounts for the difference between shifts that should be assigned to a caregiver 
            # (considering overtime hours to be balanced) and real number of shifts assigned

for nurse in I:
    expr = Y[nurse] - (HMM - nurses[nurse].accumulated_hours) * (1 / DM) <= MDH
    model += expr, f"balance_hours_1_{nurses[nurse].nurse_id}"

    expr = (HMM - nurses[nurse].accumulated_hours) * (1 / DM) - Y[nurse]<= MDH
    model += expr,  f"balance_hours_2_{nurses[nurse].nurse_id}"
    

In [146]:
# dictionary enumerating the shifts happening in each day
shifts_per_day = {}
for shift in J:
    if shifts[shift].shift_date not in shifts_per_day:
        shifts_per_day[shifts[shift].shift_date] = []
    shifts_per_day[shifts[shift].shift_date].append(shift)


In [147]:
for i in I:
    for spd in shifts_per_day.keys():
        model += plp.lpSum([X[i, j] for j in shifts_per_day[spd] if (i, j) in valid_keys]) <= 1, f"one_shift_per_day_{nurses[i].nurse_id}_{spd}"


In [148]:
# accounts for overtime V hours required
for i in I:
    model += Y[i] * DM <= (HMM - nurses[i].accumulated_hours) + V[i], f"overtime_{nurses[i].nurse_id}"

In [149]:
# demand is satisfied
for j in J:
    model += plp.lpSum([X[(i, j)] for i in I if (i, j) in valid_keys]) >= shifts[j].demand, f"demand_{j}"
        

In [150]:
# for each weekend, a nurse works at most one day
for i in I:
    for j1 in J:
        if (i, j1) in valid_keys and datetime.strptime(shifts[j1].shift_date,'%Y-%m-%d').weekday() == 5:
            model += X[i, j1] + plp.lpSum([X[i, j2] for j2 in J if
                                                        (i, j2) in valid_keys and datetime.strptime(shifts[j2].shift_date, '%Y-%m-%d').weekday() == 6
                                                        and datetime.strptime(shifts[j2].shift_date,'%Y-%m-%d') == datetime.strptime(shifts[j1].shift_date, '%Y-%m-%d') + timedelta(days=1)]) <= 1, f"weekend_{nurses[i].nurse_id}_{j1}"


In [151]:
for i,k in product(I, K):
    model += (1 - plp.lpSum([X[i, j] for j in J if (i, j) in valid_keys and datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day == k])) ==W[i, k], f"day_is_off_{nurses[i].nurse_id}_{k}"

In [152]:
days = list([datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day for j in J if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').weekday() in [5, 6]])

total_weekend_days = len(days)

for i in I:
    model += DOff[i] == plp.lpSum([W[i, k] for k in days]), f"weekend_off_1_{nurses[i].nurse_id}"
    model += DOff[i] >= floor(total_weekend_days / 2) + 1, f"weekend_off_2_{nurses[i].nurse_id}"

In [153]:
for i,w in product(I, week):
    model += Zmorning[i, w] + Zafternoon[i, w] <= 1, f"shifts_per_week_{nurses[i].nurse_id}_{w}"

In [154]:
# if a nurse works in a morning shift the varriable Z_morning is equal to 1
for i in I:
    for j in J:
        if (i, j) in valid_keys and shifts[j].shift_type == 'morning':
            model +=  X[i, j]<=Zmorning[i, datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').isocalendar()[1]], f"morning_shift_{nurses[i].nurse_id}_{j}"
        elif (i, j) in valid_keys and shifts[j].shift_type == 'afternoon':
            model +=  X[i, j]<=Zafternoon[i, datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').isocalendar()[1]], f"afternoon_shift_{nurses[i].nurse_id}_{j}"

In [155]:
#an afternoon shift cannot be followed by a morning shift
for i in I:
    for j in J:
        if (i, j) in valid_keys and shifts[j].shift_type == 'morning':
            model +=  X[i, j] + plp.lpSum([X[i, j2] for j2 in J if
                                                        (i, j2) in valid_keys and datetime.strptime(shifts[j2].shift_date, '%Y-%m-%d') == datetime.strptime(shifts[j].shift_date, '%Y-%m-%d') - timedelta(days=1)
                                                        and shifts[j2].shift_type == 'afternoon']) <= 1, f"morning_afternoon_{nurses[i].nurse_id}_{j}"

In [156]:
import numpy as np
model+= expr_PDL+expr_PWE<=np.inf, "value_obj_1"
model+= Obj_2<=np.inf , "value_obj_2"
#
model.setObjective(expr_PDL+expr_PWE)
# solve the problem using HIGHS
solver = plp.GUROBI_CMD(msg=0)
model.solve(solver)
best_1=expr_PDL.value()+expr_PWE.value()
ctr = model.constraints["value_obj_1"]
ctr.changeRHS(best_1 )
model.setObjective(Obj_2)
model.solve(solver)
worst_2=Obj_2.value()
ctr = model.constraints["value_obj_1"]
ctr.changeRHS(np.inf)
ctr = model.constraints["value_obj_2"]
ctr.changeRHS(np.inf) 
model.setObjective(Obj_2)
model.solve(solver)
best_2=Obj_2.value()
ctr = model.constraints["value_obj_2"]
ctr.changeRHS(best_2 )

model.setObjective(expr_PDL+expr_PWE)
model.solve(solver)
worst_1=expr_PDL.value()+expr_PWE.value()
print(best_1,best_2,worst_1,worst_2)

ctr = model.constraints["value_obj_1"]
ctr.changeRHS(best_1)
ctr = model.constraints["value_obj_2"]
ctr.changeRHS(np.inf)
model.setObjective(Obj_2)


7.0 7.625 8.0 8.625


In [157]:

epsilon = 1
model.solve()

while model.status == 1:
    model.solve()
    print(expr_PDL.value()+expr_PWE.value(),Obj_2.value())
    epsilon += 1
    ctr = model.constraints["value_obj_1"]
    ctr.changeRHS(best_1 + epsilon)


    if best_1 + epsilon > worst_1:
        break
    
model.setObjective(Obj_2)

7.0 8.625


In [158]:
#Get gurobi model status code
#set gurobi verbose to 1
solver = plp.GUROBI_CMD(msg=0)
model.solve(solver)

1

In [159]:
import pandas as pd
import numpy as np
"""for i in I:
    for j in J:
        if (i, j) in valid_keys and X[i, j].varValue > 0.5:
            if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day in [1,7,8,14,15,21,22,28,29] and shifts[j].shift_type!=nurses[i].shift_preference:
                print(f"Nurse {nurses[i].nurse_name} works in shift {shifts[j].shift_id} on {shifts[j].shift_date} ({shifts[j].shift_type}--{shifts[j].shift}--{nurses[i].shift_preference}) with vacation {nurses[i].vacations} and days off {nurses[i].dates_off}.")
            if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day in nurses[i].dates_off:
                print(f"Nurse {nurses[i].nurse_name} works in shift {shifts[j].shift_id} on {shifts[j].shift_date} ({shifts[j].shift_type}--{shifts[j].shift}--{nurses[i].shift_preference}) with vacation {nurses[i].vacations} and days off {nurses[i].dates_off}.")
"""
date_range=np.sort(list(set([shifts[j].shift_date for j in range(len(shifts))])))
nurses_shifts = {nurses[i].nurse_name: {shifts[j].shift_date:shifts[j].shift+" - "+shifts[j].shift_type for j in J if (i,j) in valid_keys and X[i, j].varValue > 0.5} for i in I}          
nurses_morning_shifts={nurses[i].nurse_name: {shifts[j].shift_date:shifts[j].shift for j in J if (i,j) in valid_keys and X[i, j].varValue > 0.5 and shifts[j].shift_type=="morning"} for i in I}
nurses_afternoo_shifts={nurses[i].nurse_name: {shifts[j].shift_date:shifts[j].shift for j in J if (i,j) in valid_keys and X[i, j].varValue > 0.5 and shifts[j].shift_type=="afternoon"} for i in I}

tabulated_shifts=[]
for nurse in nurses_shifts:
    line=[nurse]
    for date in date_range:
        if date not in nurses_shifts[nurse]:
            line.append("-")
            line.append("-")
        elif date in nurses_morning_shifts[nurse]:
           line.append(nurses_morning_shifts[nurse][date])
           line.append("-")
        else:
           line.append("-")
           line.append(nurses_afternoo_shifts[nurse][date])
    tabulated_shifts.append(line)
tabulated_shifts=pd.DataFrame(tabulated_shifts, columns=["Nurse"]+[str(date)+str(jornada) for date,jornada in product(date_range,["-mañana","-tarde"])])

In [160]:

tabulated_shifts.to_excel("tabulated_shifts.xlsx")
tabulated_shifts

,Nurse,2022-12-01-mañana,2022-12-01-tarde,2022-12-02-mañana,2022-12-02-tarde,2022-12-03-mañana,2022-12-03-tarde,2022-12-04-mañana,2022-12-04-tarde,2022-12-05-mañana,...,2022-12-27-mañana,2022-12-27-tarde,2022-12-28-mañana,2022-12-28-tarde,2022-12-29-mañana,2022-12-29-tarde,2022-12-30-mañana,2022-12-30-tarde,2022-12-31-mañana,2022-12-31-tarde
0,Nurse 0,-,T1,-,CHX,-,T1,-,-,M,...,M,-,M,-,M,-,M,-,-,-
1,Nurse 1,-,-,-,-,-,-,M,-,M,...,-,T2,-,T1,-,CHX,-,CHX,-,-
2,Nurse 2,COM,-,COM,-,DISP,-,-,-,COM,...,M,-,M,-,M,-,COM,-,DISP,-
3,Nurse 3,-,CHX,-,T1,-,-,-,T1,-,...,-,-,COM,-,M,-,-,-,-,-
4,Nurse 4,-,T2,-,T2,-,-,-,-,-,...,COM,-,-,-,COM,-,COM,-,M,-
5,Nurse 5,-,-,-,-,-,-,-,-,-,...,-,T1,-,CHX,-,T2,-,T2,-,T1
6,Nurse 6,M,-,M,-,-,-,-,-,-,...,M,-,-,-,COM,-,M,-,-,-
7,Nurse 7,M,-,M,-,-,-,DISP,-,-,...,-,CHX,-,T2,-,T1,-,T1,-,-
8,Nurse 8,-,-,-,-,M,-,-,-,-,...,-,-,COM,-,-,-,-,-,-,-
